In [3]:
import openai

openai.api_key = 'sk-XXXX'

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

### Final solution for stage 1 of usecase:
- RetrievalQA chain uses load_qa_chain under the hood, retrieving relevant chunks of text and feeding those to the language model

In [4]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader('./qa_docs/', glob="**/*.txt")

documents = loader.load()

In [5]:
documents[:2]

[Document(page_content='\n  \n    \n      Wir empfehlen Ihnen\n      die Kartensperrhotline anzurufen. Auf Ihren Wunsch hin werden wir die\n    \n    \n      digitale Debitkarte Ihres Smartphones sperren\n      um Missbrauch zu verhindern.\n    \n  \n', metadata={'source': 'qa_docs/qa.40.txt'}),
 Document(page_content='WARUM IST RAIPAY NICHT MIT DEM ENTWICKLER:INNENMODUS AUF MEINEM SMARTPHONE KOMPATIBEL? Die aktuellen Sicherheitseinstellungen der RaiPay App erlauben keine App Nutzung mit aktiviertem Entwickler:innenmodus am Smartphone. Wir arbeiten aber bereits an einer zukünftigen Aufhebung dieser Einschränkung um das RaiPay Nutzererlebnis für unsere Kund:innen noch weiter zu verbessern.', metadata={'source': 'qa_docs/qa.33.txt'})]

In [5]:
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

In [ ]:
vectorstore = Chroma.from_documents(documents, embeddings, persist_directory='./chroma_storage')

In [39]:
vectorstore.persist()
vectorstore = None

In [6]:
# Now we can load the persisted database from disk, and use it as normal. 
vectorstore = Chroma(persist_directory='./chroma_storage/', embedding_function=embeddings)


In [7]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True,
    input_key='question', output_key='answer'
)

In [8]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0, openai_api_key=openai.api_key), 
    vectorstore.as_retriever(), 
    memory=memory,
    return_source_documents=True
)

In [9]:
query = "Kannen sie im ausland RAIPAY nutze?"
result = qa({"question": query})
result

{'question': 'Kannen sie im ausland RAIPAY nutze?',
 'chat_history': [HumanMessage(content='Kannen sie im ausland RAIPAY nutze?', additional_kwargs={}, example=False),
  AIMessage(content=' Ja, Sie können mit RaiPay auch im Ausland an Terminals kontaktlos bezahlen oder Bargeld an Geldautomaten abheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Das Verfahren ist das Gleiche wie in Österreich.', additional_kwargs={}, example=False)],
 'answer': ' Ja, Sie können mit RaiPay auch im Ausland an Terminals kontaktlos bezahlen oder Bargeld an Geldautomaten abheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Das Verfahren ist das Gleiche wie in Österreich.',
 'source_documents': [Document(page_content='KANN ICH MIT RAIPAY AUCH IM AUSLAND ZAHLEN? Ja, Sie können mit RaiPay auch im Ausland an Terminals kontaktlos bezahlen oder Bargeld an Geldautomaten abheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Das Verfahren ist das Gleiche wie in Österreich.', metada

In [17]:
query = "Und giebt es einen limit fur zahlung?"
result = qa({"question": query})
result

{'question': 'Und giebt es einen limit fur zahlung?',
 'chat_history': [HumanMessage(content='Kannen sie im ausland RAIPAY nutze?', additional_kwargs={}, example=False),
  AIMessage(content=' Ja, Sie können mit RaiPay auch im Ausland an Terminals kontaktlos bezahlen oder Bargeld an Geldautomaten abheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Das Verfahren ist das Gleiche wie in Österreich.', additional_kwargs={}, example=False),
  HumanMessage(content='Kannen sie im ausland RAIPAY nutze?', additional_kwargs={}, example=False),
  AIMessage(content=' Ja, Sie können mit RaiPay auch im Ausland an Terminals kontaktlos bezahlen oder Bargeld an Geldautomaten abheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Das Verfahren ist das Gleiche wie in Österreich.', additional_kwargs={}, example=False),
  HumanMessage(content='Kannen sie im ausland RAIPAY nutze?', additional_kwargs={}, example=False),
  AIMessage(content=' Ja, Sie können mit RaiPay auch im Ausland an T

In [21]:
query = "Vie viele baren sind im Slowakei?"
result = qa({"question": query, 'vectordbkwargs': {"search_distance": 0.99}})
result

{'question': 'Vie viele baren sind im Slowakei?',
 'vectordbkwargs': {'search_distance': 0.99},
 'chat_history': [HumanMessage(content='Kannen sie im ausland RAIPAY nutze?', additional_kwargs={}, example=False),
  AIMessage(content=' Ja, Sie können mit RaiPay auch im Ausland an Terminals kontaktlos bezahlen oder Bargeld an Geldautomaten abheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Das Verfahren ist das Gleiche wie in Österreich.', additional_kwargs={}, example=False),
  HumanMessage(content='Kannen sie im ausland RAIPAY nutze?', additional_kwargs={}, example=False),
  AIMessage(content=' Ja, Sie können mit RaiPay auch im Ausland an Terminals kontaktlos bezahlen oder Bargeld an Geldautomaten abheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Das Verfahren ist das Gleiche wie in Österreich.', additional_kwargs={}, example=False),
  HumanMessage(content='Kannen sie im ausland RAIPAY nutze?', additional_kwargs={}, example=False),
  AIMessage(content=' Ja, 

In [26]:
query = "Vie viele Bären sind im Slowakei?"
result = qa({"question": query, 'vectordbkwargs': {"search_distance": 0.2}})
print(
    result['answer'],
    len(result['source_documents'])
)

 Es ist möglich, mit RaiPay im Ausland an Terminals kontaktlos zu bezahlen oder Bargeld an Geldautomaten abzuheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Es gibt keine Einschränkungen hinsichtlich der Anzahl der Bargeldabhebungen in der Slowakei. 4


In [29]:
query = "Vie viele Baren sind im Slowakei?"
result = qa({"question": query, 'vectordbkwargs': {"search_distance": 0.9}})
print(
    result['answer'],
    len(result['source_documents'])
)

 Es ist möglich, mit RaiPay im Ausland an Terminals kontaktlos zu bezahlen oder Bargeld an Geldautomaten abzuheben, die mit einem kontaktlosen Lesegerät ausgestattet sind. Es gibt jedoch keine Informationen darüber, wie viele Bargeldabhebungen in der Slowakei mit RaiPay möglich sind. 4
